# Explicación de la notebook 4

En esta notebook se separan las variables en categóricas y continuas. Las continuas, seran normalizadas mediante la resta de la media y división por el desvio para mejorar luego el entrenamiento. A las variables categóricas que no entén en formato numérico o booleano, se les realiza un "encoding" para pasarlas a numericas y poder utilizarlas para la regresión (se cambio la selección de categoricas y continuas dadas por default en la notebook y, con estos cambios, se consiguieron mejores resultados en las siguientes notebooks. Por ejemplo, "promo" o "Schoolholiday" esta seleccionada como continua y se le realizaba unan normalización antes pero solo toma valores de 0 o 1, por lo que se decidio tomarla como categorica). Además, como en las otras notebooks, se corrigio código que tiraba mensajes de error, como por ejemplo mas adelante que se usaba la variable StateHoliday_fw, que ya no existía porque se la reemplazo por StateHoliday_bool_fw.

Se eligieron como variables categoricas a:
            
            'Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 
            'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 
            'CompetitionOpenSinceYear', 'Promo2SinceYear', 
            'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 
            'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 
            'SchoolHoliday_bw', 'Promo', 'SchoolHoliday', 'StateHoliday_bool'

Se eligieron como continuas a :

                'CompetitionDistance', 
               'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 
               'Precipitationmm', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
               'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 
               'trend_DE', 'AfterStateHoliday_bool', 'BeforeStateHoliday_bool'

# Código

In [42]:
import pandas as pd
import numpy as np
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import LabelEncoder, StandardScaler
from matplotlib import pyplot as plt

In [2]:
# pip install pyarrow

In [40]:
df_train = pd.read_feather('all_preprocessed_train.fth')
df_test = pd.read_feather('all_preprocessed_test.fth')

In [44]:
df_train[df_train['Store']==280]['Sales'].values[100:310]

array([ 4538,  4656,  4908,     0,  6630,  7202,  6934,  6083,  6073,
        8099,     0,  5711,  5504,  4854,  4564,  5856,     0,     0,
        8713,     0,  9945,  8767,  7260,  8938,     0,  6056,  6345,
        5853,  4322,  4955,  5211,     0,  5424,  6821,  6212,  5294,
        6653,  8224,     0,  6021,  5768,  5864,  4519,  5211,  5689,
           0,  5348,  6688,  5922,  6162,  6911,  7892,     0,  6458,
        6409,  5638,  4645,  5143,  5655,     0,  6410,  7218,  6638,
        6198,  1673,  7002,     0,  6820,  6459,  3867,  6126,  5434,
        5763,     0,  6211,  6682,  6893,  5660,  7532,  4414,     0,
        6563,  6275,  5555,  5884,  5260,  7090,     0,  5768,  4902,
        5128,  4959,  3749,  4853,     0,  6032,  8078,  6695,  5695,
        6596,  7665,     0,  5821,  7724,  7053,  7529,     0,  9946,
           0,  5967,  5248,     0,  4176,  5925,  5442,     0,  5028,
           0,     0,  6526, 12120, 11011,     0,  9140,  9921,  9981,
        8839,  9159,

In [4]:
df_train.shape, df_test.shape

((1017209, 84), (41088, 83))

In [5]:
# Verificación de que se hicieron bien las cosas en testa también
set(df_train.columns) - set(df_test.columns)

{'Customers', 'Sales'}

In [6]:
(df_train['CompetitionDistance'].isna()).sum()

0

In [54]:
df_train['StateHoliday'].value_counts()


0    986159
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64

## Categorical vars

In [56]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 
            'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 
            'CompetitionOpenSinceYear', 'Promo2SinceYear', 
            'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 
            'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 
            'SchoolHoliday_bw', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']

In [ ]:
#cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 
#            'CompetitionMonthsOpen', 'Promo2Weeks', 
#            'StoreType', 'Assortment', 'PromoInterval', 
#            'CompetitionOpenSinceYear', 'Promo2SinceYear', 
#            'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 
#            'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 
#            'SchoolHoliday_bw']

In [9]:
#DataFrameSummary(df_train[cat_vars]).summary()

In [10]:
DataFrameSummary(df_train[cat_vars]).summary().loc[['uniques', 'types', 'missing']].T

,uniques,types,missing
Store,1115,numeric,0
DayOfWeek,7,numeric,0
Year,3,numeric,0
Month,12,numeric,0
Day,31,numeric,0
StateHoliday,4,categorical,0
CompetitionMonthsOpen,25,numeric,0
Promo2Weeks,26,numeric,0
StoreType,4,categorical,0
Assortment,3,categorical,0


In [11]:
# Reemplaza nulls por "" en categoricas
# for v in cat_vars: df_train.loc[df_train[v].isnull(), v] = ""

In [12]:
# for v in cat_vars: df_test.loc[df_test[v].isnull(), v] = ""

## Continuos Vars

In [13]:
df_train.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
       'CompetitionOpenSince', 'Promo2Since', 'State', 'file', 'week', 'trend',
       'Date_y', 'Month_y', 'Day_y', 'file_DE', 'week_DE', 'trend_DE',
       'Date_DE', 'State_DE', 'Month_DE', 'Day_DE', 'file_y',
       'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
       'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC', 'Max_Humidity',
       'Mean_Humidity', 'Min_Humidity', 'Max_Sea_Level_PressurehPa',
       'Mean_Sea_Level_PressurehPa', 'Min_Sea_Level_PressurehPa',
       'Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM',
       'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'Max_Gust_SpeedKm_h',
       'Precipitationmm', 'Cl

In [58]:
contin_vars = ['CompetitionDistance', 
               'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 
               'Precipitationmm', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
               'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 
               'trend_DE', 'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 
               ]

In [59]:
#contin_vars = ['CompetitionDistance', 
#               'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 
#               'Precipitationmm', 'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 
#               'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 
#               'trend_DE', 'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 
#               'Promo', 'SchoolHoliday', 'StateHoliday_bool']

In [60]:
DataFrameSummary(df_train[contin_vars]).summary().loc[['uniques', 'types', 'missing']].T

,uniques,types,missing
CompetitionDistance,654,numeric,0
Max_TemperatureC,50,numeric,0
Mean_TemperatureC,45,numeric,0
Min_TemperatureC,40,numeric,0
Precipitationmm,40,numeric,0
Max_Humidity,52,numeric,0
Mean_Humidity,71,numeric,0
Min_Humidity,93,numeric,0
Max_Wind_SpeedKm_h,42,numeric,0
Mean_Wind_SpeedKm_h,27,numeric,0


In [61]:
# Reemplaza nulls por 0 en continuas
# for v in contin_vars: df_train.loc[df_train[v].isnull(), v] = 0
# for v in contin_vars: df_test.loc[df_test[v].isnull(), v] = 0

## Normalización en continuas y LabelEncode en Categoricas

In [62]:
# pip install sklearn_pandas
from sklearn_pandas import DataFrameMapper

In [63]:
cat_maps = [(o, LabelEncoder()) for o in cat_vars]
contin_maps = [([o], StandardScaler()) for o in contin_vars]

In [64]:
mapper_cat = DataFrameMapper(cat_maps)
_ = mapper_cat.fit(df_train)

In [65]:
# Assortment es la novena posición en cat_maps
N = 10
print(list(zip(df_train['Assortment'].values[:N], mapper_cat.transform(df_train)[:,9][:N])))
print(list(zip(df_train['Events'].values[:N], mapper_cat.transform(df_train)[:,15][:N])))
print(list(zip(df_train['Year'].values[:N], mapper_cat.transform(df_train)[:,2][:N])))

[('a', 0), ('a', 0), ('a', 0), ('c', 2), ('a', 0), ('a', 0), ('c', 2), ('a', 0), ('c', 2), ('a', 0)]
[('Fog', 0), ('Fog', 0), ('Fog', 0), ('Sunny', 20), ('Sunny', 20), ('Sunny', 20), ('Rain', 10), ('Rain', 10), ('Fog', 0), ('Sunny', 20)]
[(2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2), (2015, 2)]


In [66]:
mapper_cont = DataFrameMapper(contin_maps)
_ = mapper_cont.fit(df_train)

In [67]:
N = 10
print(df_train['CompetitionDistance'].values[:N])
print(mapper_cont.transform(df_train)[:, 0][:N])

[ 1270.   570. 14130.   620. 29910.   310. 24000.  7520.  2030.  3160.]
[-0.51104292 -0.59341201  1.00219483 -0.5875285   2.85902934 -0.62400624
  2.16359893  0.22439533 -0.42161363 -0.28864639]


In [68]:
# La hacemos con todas ahora
df_train[cat_vars] = mapper_cat.transform(df_train)
df_test[cat_vars] = mapper_cat.transform(df_test)

In [69]:
df_train[contin_vars] = mapper_cont.transform(df_train)
df_test[contin_vars] = mapper_cont.transform(df_test)

In [70]:
df_train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,AfterStateHoliday_bool,BeforeStateHoliday_bool,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bool_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_bool_fw,Promo_fw
0,0,4,2015-07-31,5263,555,1,1,0,1,2,...,0.644376,1.072424,0,0,5,0,5,1,0,1
1,1,4,2015-07-31,6064,625,1,1,0,1,2,...,0.965073,1.072424,0,0,5,0,5,1,0,1
2,2,4,2015-07-31,8314,821,1,1,0,1,2,...,0.644376,1.072424,0,0,5,0,5,1,0,1
3,3,4,2015-07-31,13995,1498,1,1,0,1,2,...,0.965073,1.072424,0,0,5,0,5,1,0,1
4,4,4,2015-07-31,4822,559,1,1,0,1,2,...,0.644376,1.072424,0,0,5,0,5,1,0,1


In [71]:
DataFrameSummary(df_train[cat_vars]).summary().loc[['uniques', 'types']]

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Events,Promo_fw,Promo_bw,StateHoliday_bool_fw,StateHoliday_bool_bw,SchoolHoliday_fw,SchoolHoliday_bw,Promo,SchoolHoliday,StateHoliday_bool
uniques,1115,7,3,12,31,4,25,26,4,3,...,22,6,6,3,3,8,8,2,2,2
types,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,...,numeric,numeric,numeric,numeric,numeric,numeric,numeric,bool,bool,bool


## Final processing

In [72]:
df_train.shape

(1017209, 84)

In [73]:
df_train.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval',
       'CompetitionOpenSince', 'Promo2Since', 'State', 'file', 'week', 'trend',
       'Date_y', 'Month_y', 'Day_y', 'file_DE', 'week_DE', 'trend_DE',
       'Date_DE', 'State_DE', 'Month_DE', 'Day_DE', 'file_y',
       'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
       'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC', 'Max_Humidity',
       'Mean_Humidity', 'Min_Humidity', 'Max_Sea_Level_PressurehPa',
       'Mean_Sea_Level_PressurehPa', 'Min_Sea_Level_PressurehPa',
       'Max_VisibilityKm', 'Mean_VisibilityKm', 'Min_VisibilitykM',
       'Max_Wind_SpeedKm_h', 'Mean_Wind_SpeedKm_h', 'Max_Gust_SpeedKm_h',
       'Precipitationmm', 'Cl

In [74]:
df_sales = df_train[df_train.Sales!=0]

In [75]:
df_sales.shape

(844338, 84)

In [76]:
df_test.reset_index(inplace=True)

In [77]:
df_sales.reset_index(inplace=True)

In [78]:
df_sales.to_feather('train_normalized_data.fth')

In [79]:
df_test.to_feather('test_normalized_data.fth')

In [80]:
(df_test['Open'] == 0).sum()

5984

In [81]:
(df_sales['Open'] == 0).sum()

0

In [82]:
(df_train['CompetitionDistance'].isna()).sum()

0

In [85]:
DataFrameSummary(df_train[cat_vars]).summary().loc[['uniques', 'types', 'missing']].T

,uniques,types,missing
Store,1115,numeric,0
DayOfWeek,7,numeric,0
Year,3,numeric,0
Month,12,numeric,0
Day,31,numeric,0
StateHoliday,4,numeric,0
CompetitionMonthsOpen,25,numeric,0
Promo2Weeks,26,numeric,0
StoreType,4,numeric,0
Assortment,3,numeric,0
